In [1]:
!pip install -U unstructured
!pip install llama-index
!pip install llama-index-llms-openai
!pip install pyvis
!pip install arize-phoenix[evals]
!pip install llama-index-callbacks-arize-phoenix

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 981.5/981.5 kB 15.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 62.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 431.4/431.4 kB 17.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 274.7/274.7 kB 15.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.4/3.4 MB 67.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.1/45.1 kB 2.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 82.7/82.7 kB 4.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.4/76.4 kB 4.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 4.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.3/49.3 kB 2.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 295.8/295.8 kB 15.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.5/54.5 kB 2.8 MB/s eta 0:00:00
   ━━━━━━━━

In [2]:
import json
from typing import Sequence, List

from llama_index.llms.openai import OpenAI
from llama_index.core.llms import ChatMessage
from llama_index.core.tools import BaseTool, FunctionTool

import nest_asyncio

nest_asyncio.apply()

import os

os.environ["OPENAI_API_KEY"] = ""






In [3]:
def multiply(a: int, b: int) -> int:
    "Multiply two integers and return an integer"
    return a * b

multiply_tool = FunctionTool.from_defaults(fn=multiply)


def add(a: int, b: int) -> int:
    "Add two integers and return an integer"
    return a + b

add_tool = FunctionTool.from_defaults(fn=add)



In [4]:
tools = [multiply_tool, add_tool]

llm = OpenAI(model="gpt-4o")

from llama_index.core.agent import AgentRunner
from llama_index.agent.openai import OpenAIAgent, OpenAIAgentWorker

agent = OpenAIAgent.from_tools(tools, llm=llm, verbose=True)



In [5]:
response = agent.chat("What is (178 * 3) + 98?")
print(response)



Added user message to memory: What is (178 * 3) + 98?
=== Calling Function ===
Calling function: multiply with args: {"a": 178, "b": 3}
Got output: 534

=== Calling Function ===
Calling function: add with args: {"a": 0, "b": 98}
Got output: 98

=== Calling Function ===
Calling function: add with args: {"a":534,"b":98}
Got output: 632

The result of \((178 \times 3) + 98\) is \(632\).


In [6]:
task = agent.create_task("What is (178 * 3) + 98?")
step_output = agent.run_step(task.task_id)
print(step_output)

Added user message to memory: What is (178 * 3) + 98?
The result of \((178 \times 3) + 98\) is \(632\).


In [7]:
step_output.is_last

True

In [8]:
response = agent.finalize_response(task.task_id)
print(str(response))

The result of \((178 \times 3) + 98\) is \(632\).


In [9]:
llm = OpenAI(model="gpt-4o")

In [10]:
from llama_index.core.agent import AgentRunner, ReActAgentWorker, ReActAgent


In [11]:
agent = ReActAgent.from_tools(tools, llm=llm, verbose=True)

In [12]:
agent.chat("Hey!")

> Running step dabae37b-7fd5-4814-aad8-79225922256c. Step input: Hey!
Thought: The current language of the user is: English. The user has greeted me, and I should respond accordingly.
Answer: Hello! How can I assist you today?


AgentChatResponse(response='Hello! How can I assist you today?', sources=[], source_nodes=[], is_dummy_stream=False, metadata=None)

In [13]:
response = agent.chat("What is (121 * 3) + 42?")

> Running step 44a92bab-d8da-411a-bbb3-aa3f20e41f5b. Step input: What is (121 * 3) + 42?
Thought: The current language of the user is English. I need to use tools to calculate the result of the expression (121 * 3) + 42.
Action: multiply
Action Input: {'a': 121, 'b': 3}
Observation: 363
> Running step f3a6fd12-7e3d-41d8-bd2f-7a081141de54. Step input: None
Thought: I have the result of the multiplication part of the expression. Now, I need to add 42 to 363.
Action: add
Action Input: {'a': 363, 'b': 42}
Observation: 405
> Running step 69ce7d6e-158b-4959-8922-d2cf372e6802. Step input: None
Thought: I have the final result of the expression.
Answer: The result of (121 * 3) + 42 is 405.


In [14]:
response

AgentChatResponse(response='The result of (121 * 3) + 42 is 405.', sources=[ToolOutput(content='363', tool_name='multiply', raw_input={'args': (), 'kwargs': {'a': 121, 'b': 3}}, raw_output=363, is_error=False), ToolOutput(content='405', tool_name='add', raw_input={'args': (), 'kwargs': {'a': 363, 'b': 42}}, raw_output=405, is_error=False)], source_nodes=[], is_dummy_stream=False, metadata=None)

In [15]:
task = agent.create_task("What is (121 * 3) + 42?")

In [16]:
step_output = agent.run_step(task.task_id)

> Running step 8ce2b38c-6b5f-4f50-8c65-0763f78712b5. Step input: What is (121 * 3) + 42?
Thought: The current language of the user is English. I need to use a tool to help me answer the question.
Action: multiply
Action Input: {'a': 121, 'b': 3}
Observation: 363


In [17]:
step_output.output

AgentChatResponse(response='Observation: 363', sources=[ToolOutput(content='363', tool_name='multiply', raw_input={'args': (), 'kwargs': {'a': 121, 'b': 3}}, raw_output=363, is_error=False)], source_nodes=[], is_dummy_stream=False, metadata=None)

In [18]:
step_output = agent.run_step(task.task_id)

> Running step c52c0997-33ee-47e9-81eb-ed6188e5c458. Step input: None
Thought: I have the result of the multiplication part. Now I need to add 42 to it.
Action: add
Action Input: {'a': 363, 'b': 42}
Observation: 405


In [19]:
step_output = agent.run_step(task.task_id)

> Running step 83e5431b-bff5-4c7a-96a0-d1f9072b602f. Step input: None
Thought: I can answer without using any more tools. I'll use the user's language to answer.
Answer: The result of (121 * 3) + 42 is 405.
